In this notebook we will optimize parameters of the rating system

## Imports

In [33]:
from scraping import ufcstats
from glicko2 import Fighter
import pandas as pd

## Data

In [2]:
fights = ufcstats.get_completed_fights()

In [6]:
fights_df = pd.DataFrame(fights)
fights_df.head()

,event,date,fighter,opposition,weight_class,outcome,method
0,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Dricus Du Plessis,Sean Strickland,Middleweight,win,U-DEC
1,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Zhang Weili,Tatiana Suarez,Women's Strawweight,win,U-DEC
2,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Tallison Teixeira,Justin Tafa,Heavyweight,win,KO/TKO
3,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Jimmy Crute,Rodolfo Bellato,Light Heavyweight,draw,M-DEC
4,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Jake Matthews,Francisco Prado,Welterweight,win,U-DEC


In [ ]:
fights_df['outcome'].unique()

array(['win', 'draw', 'nc'], dtype=object)

In [12]:
fights_df['method'].unique()

array(['U-DEC', 'KO/TKO', 'M-DEC', 'SUB', 'S-DEC', 'CNC', 'DQ',
       'Overturned', 'Other'], dtype=object)

In [21]:
fights_df[fights_df['outcome'] == 'draw']

,event,date,fighter,opposition,weight_class,outcome,method
3,UFC 312: Du Plessis vs. Strickland 2,2025-02-08,Jimmy Crute,Rodolfo Bellato,Light Heavyweight,draw,M-DEC
624,UFC 295: Prochazka vs. Pereira,2023-11-11,Nazim Sadykhov,Viacheslav Borshchev,Lightweight,draw,M-DEC
634,UFC Fight Night: Almeida vs. Lewis,2023-11-04,Rinat Fakhretdinov,Elizeu Zaleski dos Santos,Welterweight,draw,M-DEC
684,UFC Fight Night: Grasso vs. Shevchenko 2,2023-09-16,Alexa Grasso,Valentina Shevchenko,Women's Flyweight,draw,S-DEC
987,UFC 286: Edwards vs. Usman 3,2023-03-18,Jai Herbert,Ludovit Klein,Lightweight,draw,M-DEC
1041,UFC 284: Makhachev vs. Volkanovski,2023-02-11,Jimmy Crute,Alonzo Menifield,Light Heavyweight,draw,M-DEC
1053,UFC Fight Night: Lewis vs. Spivac,2023-02-04,Dooho Choi,Kyle Nelson,Featherweight,draw,M-DEC
1100,UFC 282: Blachowicz vs. Ankalaev,2022-12-10,Jan Blachowicz,Magomed Ankalaev,Light Heavyweight,draw,S-DEC
1252,UFC 278: Usman vs. Edwards,2022-08-20,Sean Woodson,Luis Saldana,Featherweight,draw,S-DEC
1269,UFC Fight Night: Vera vs. Cruz,2022-08-13,Youssef Zalal,Da'Mon Blackshear,Bantamweight,draw,M-DEC


In [32]:
SCORING_DICT = {'win': {'KO/TKO': 1, 'SUB': 1, 'U-DEC': 0.95, 'M-DEC': 0.925, 'S-DEC': 0.9, 'DQ': 0.6},
                'loss': {'KO/TKO': 0, 'SUB': 0, 'U-DEC': 0.05, 'M-DEC': 0.075, 'S-DEC': 0.1, 'DQ': 0.4},
                'draw': 0.5}